In [264]:
import os
import fiona
import geemap
import numpy as np
import pandas as pd
import geopandas as gpd
import xml.etree.ElementTree as ET

from shapely.geometry import shape
from shapely.geometry import Point

# Clustering (best for testing** due to speed)
from sklearn.cluster import KMeans

# tif file creation
import rasterio
from rasterio.transform import from_origin
from rasterio.features import rasterize

# Plotting and Vis 
import seaborn as sns
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm

import re

In [265]:
# Initialise
ee.Authenticate()
ee.Initialize(project="jameswilliamchamberlain")

In [266]:
# basemap 
basemap_url = 'https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}'

# centeral point of Samarra 
df_sites = pd.DataFrame({
    "longitude": [43.823543],
    "latitude": [34.340989],
    "name": ["Samarra Archaeological City"],
    "category": ["Cultural"],
    "date inscribed": ["2007"],
    "region": ["Arab States"],
    "url": ["https://whc.unesco.org/en/list/276"],
    "iso": [["IQ"]]
})

# Chunks of Samarra Archaeological City 
with fiona.open("chunks_new.shp") as src:
    chunks = gpd.GeoDataFrame.from_features(src, crs=src.crs)

In [267]:
m = geemap.Map()

if df_sites.empty:
    print("No sites found for the specified URL.")
else:
    m.add_points_from_xy(df_sites, x="longitude", y="latitude", layer_name="Sites")
    center_points = df_sites[['longitude', 'latitude']].mean().values
    m.setCenter(center_points[0], center_points[1], 10)

m.add_basemap(basemap_url, name="Google Satellite", attribution="Google")

# add chunks from aoi 
m.add_gdf(chunks, layer_name="AOI", style={"color": "red", "fillColor": "red", "fillOpacity": 0.1})

m

Map(center=[34.340989, 43.823543], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Sear…

In [268]:
def create_subregions(chunks, sift_percentage=0.5):
    """
        Shifts all polygons to 8 positions based on half the length in longitude and latitude, to create subregions - one for each direction from the center.

        Assumes aoi contains all similar polygons, and are similar to a square.

        Parameters:
            aoi (GeoDataFrame): The area of interest containing geometries.
            sift_percentage (float): Percentage of the length to shift the center point. Default is 0.5 (50%) for half the length.
    """

    if chunks.empty:
        return {}
    
    first_polygon = chunks.geometry.iloc[0]

    # take top two points of the polygon and get the length between them
    top_points = first_polygon.exterior.coords[:2]
    length_lon = abs(top_points[0][0] - top_points[1][0])

    # calculate the shift amount
    shift_amount = length_lon * sift_percentage 

    shift_directions = {
        "left": (-shift_amount, 0),
        "right": (shift_amount, 0),
        "up": (0, shift_amount),
        "down": (0, -shift_amount),
        "top_left": (-shift_amount, shift_amount),
        "top_right": (shift_amount, shift_amount),
        "bottom_left": (-shift_amount, -shift_amount),
        "bottom_right": (shift_amount, -shift_amount),
    }

    subregions = []
        
    # Create subregions by shifting the geometries in all directions
    for _, (dx, dy) in shift_directions.items():
        gdf_shifted = chunks.copy()
        gdf_shifted["geometry"] = gdf_shifted["geometry"].translate(dx, dy)
        subregions.append(gdf_shifted)

    # Combine all into one GeoDataFrame
    subregions = pd.concat(subregions, ignore_index=True)
    return gpd.GeoDataFrame(subregions, crs=chunks.crs)

def clip(chunks, aoi):
    """
        Clips the chunks or subregions to the area of interest (aoi).

        Parameters:
            chunks (GeoDataFrame):      The chunks or subregions to be clipped.
            aoi (GeoDataFrame):         The area of interest (aoi) to clip the chunks against.
    """

    # clip to aoi 
    if chunks.empty or aoi.empty:
        return gpd.GeoDataFrame(columns=chunks.columns.tolist(), crs=chunks.crs)
    clipped = gpd.clip(chunks, aoi)
    clipped = clipped[clipped.geometry.notnull()]  # Remove any null geometries

    return clipped.reset_index(drop=True)

subregions = create_subregions(chunks)
subregions = clip(subregions, aoi=chunks.dissolve())

# plot as one 
m.add_gdf(subregions, layer_name="Subregions", style={"color": "blue", "fillColor": "blue", "fillOpacity": 0.1})

# Reference Points

In [ ]:
def collect_points_from_geemap(map_obj, label):
    """
    Collect all drawn point features from a geemap.Map that uses ee.Feature objects,
    and return them as a GeoDataFrame with a label.

    Parameters:
        map_obj (geemap.Map): The interactive map.
        label (str): Label to assign to all collected points.

    Returns:
        GeoDataFrame: With geometry and 'label' columns.
    """

    features = map_obj.draw_features

    if not features:
        return gpd.GeoDataFrame(columns=["geometry", "label"], geometry="geometry")

    points = []
    for f in features:
        try:
            geom = f.geometry()  # call the method
            if geom.getInfo()["type"] == "Point":
                coords = geom.coordinates().getInfo()  # [lon, lat]
                points.append(Point(coords))
        except Exception as e:
            print("Skipping feature due to error:", e)

    if not points:
        return gpd.GeoDataFrame(columns=["geometry", "label"], geometry="geometry")

    gdf = gpd.GeoDataFrame(geometry=points)
    gdf["label"] = label
    gdf.set_crs("EPSG:4326", inplace=True)
    return gdf

def gen_basemap(basemap_url=None, aoi=gpd.GeoDataFrame(), polygons=None):
    """
        Generates a basemap with the specified URL and adds polygons if provided.

        Parameters:
            basemap_url (str): The URL of the basemap to be used.
            polygons (GeoDataFrame, optional): Polygons to be added to the map. Defaults to None.
            aoi (GeoDataFrame): The area of interest to be displayed on the map. (can be a set of polygons or a single polygon)
        
        Returns:
            geemap.Map: A geemap map object with the specified basemap and polygons.
    """
    m = geemap.Map()

    if not basemap_url:
        basemap_url = 'https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}'
        
    m.add_basemap(basemap_url, name="Google Satellite", attribution="Google")

    # center on aoi
    if not aoi.empty:
        m.add_gdf(aoi, layer_name="AOI", style={"color": "red", "fillColor": "red", "fillOpacity": 0.1})
        center_points = aoi.geometry.unary_union.centroid.coords[0]
        m.setCenter(center_points[0], center_points[1], 10)
    
    if polygons is not None:
        m.add_gdf(polygons, layer_name="Polygons", style={"color": "red", "fillColor": "red", "fillOpacity": 0.1})
    
    return m

In [ ]:
# Example of 3 maps for urban, agricultrual, water and waste
urban_map = gen_basemap(basemap_url, aoi=chunks)
agricultural_map = gen_basemap(basemap_url, aoi=chunks)
water_map = gen_basemap(basemap_url, aoi=chunks)
# wasteland_map = geemap.Map()

In [42]:
urban_map

Map(center=[34.223846605905464, 43.91319768371038], controls=(WidgetControl(options=['position', 'transparent_…

In [43]:
agricultural_map

Map(center=[34.223846605905464, 43.91319768371038], controls=(WidgetControl(options=['position', 'transparent_…

In [ ]:
# water_map = water_waste_map
water_map

Map(center=[34.223846605905464, 43.91319768371038], controls=(WidgetControl(options=['position', 'transparent_…

In [ ]:
# convert to labelled points in GeoDataFrame

water_points = collect_points_from_geemap(water_map, label="Water")
agricultural_points = collect_points_from_geemap(agricultural_map, label="Agricultural")
urban_points = collect_points_from_geemap(urban_map, label="Urban")


In [ ]:
# merge into one GeoDataFrame
points_list = [water_points, agricultural_points, urban_points]
labels = pd.concat(points_list, ignore_index=True)

141

In [56]:
urban_points.head()

,geometry,label
0,POINT (43.87502 34.20002),Urban
1,POINT (43.87279 34.19114),Urban
2,POINT (43.88326 34.18901),Urban
3,POINT (43.88763 34.19626),Urban
4,POINT (43.88635 34.20428),Urban


# Clustering
Here we create a Sparse matrix

then with the sparse matrix compare and label based on that

In [85]:
pth = "data/"

# collect paths for all csv files in the folder
paths = []

for root, dirs, files in os.walk(pth):
    for file in files:
        if file.endswith(".csv"):
            paths.append(os.path.join(root, file))

print(f"Found {len(paths)} CSV files.")

# select 2024 
path = [p for p in paths if "2019" in p]
path


Found 6 CSV files.


['data/2019.csv']

In [211]:
def prep_data(dir):
    """Creates two dataframes from a CSV file, one ready for clustering with additional columns removed and the other with all columns left intact."""
    df1 = pd.read_csv(dir)
    df1 = df1.dropna()
    df2_clear = df1.copy()
    df2_clear = df2_clear.drop(columns=["system:index", ".geo"])
    df2_clear = df2_clear.set_index("file_name")
    df1 = df1.set_index("file_name")
    df2_clear = df2_clear.apply(pd.to_numeric, errors='coerce')

    return df1, df2_clear

df1, df2_clear = prep_data(path[0])

In [ ]:
# base clustering function - and an example of how to format it
def kmeans_clustering(df, k=10):
    """sklearn kmeans"""

    kmeans = KMeans(n_clusters=k, random_state=42) # TODO: make random_state random
    kmeans.fit(df)
    
    return kmeans.labels_, df.index.tolist()

class cluster:
    """
        Takes a set of polygons and a set of points with attached geometry 

        and runs clustering over all points in the polygons, and returns a set of clusters based on the points.
    """

    df_sparse = pd.DataFrame()

    def __init__(self, subregions, df_data, aoi=None, index_column="file_name", points=gpd.GeoDataFrame()):
        self.data = df_data

        # Subregions 
        if aoi is None:
            aoi = subregions.dissolve()

        subregions_poly = create_subregions(chunks)
        subregions_poly = clip(subregions, aoi=aoi)
        self.subregions = subregions_poly

        self.index_column = index_column
        self.points = points
        self.UTM_ESPG = 32638
        self.EPSG = 4326

        self.gdf_labels = None
        self.df_sparse = pd.DataFrame()
        

    def __fit__(self):
        """
            Fits the clustering model to the data.
            This method should be implemented in subclasses.
        """
        raise NotImplementedError("Subclasses should implement this method.")
    
    def clip_dataframe(self, polygon, df):
        """
            Clips the DataFrame based on the .geo column and a given polygon.   
        
        Args:
            polygon (shapely.geometry.Polygon): The polygon to clip the DataFrame to.
            df (pd.DataFrame): The DataFrame containing the geometries to be clipped.
            
        Returns:
            pd.DataFrame: The clipped DataFrame.
        """

        # based on .geo column drop all rows that do not intersect with the polygon
        df['geometry'] = df['.geo'].apply(lambda x: shape(json.loads(x)))
        gdf = gpd.GeoDataFrame(df, geometry='geometry', crs=self.UTM_ESPG)
        gdf = gdf[gdf.geometry.within(polygon)]
        gdf = gdf.drop(columns=['geometry'])

        # convert back to DataFrame
        df_clipped = pd.DataFrame(gdf)

        return df_clipped
    
    def clip_polygon(chunks, aoi):
        """
            Clips the chunks or subregions to the area of interest (aoi).

            Parameters:
                chunks (GeoDataFrame):      The chunks or subregions to be clipped.
                aoi (GeoDataFrame):         The area of interest (aoi) to clip the chunks against.
        """

        # clip to aoi 
        if chunks.empty or aoi.empty:
            return gpd.GeoDataFrame(columns=chunks.columns.tolist(), crs=chunks.crs)
        clipped = gpd.clip(chunks, aoi)
        clipped = clipped[clipped.geometry.notnull()]  # Remove any null geometries

        return clipped.reset_index(drop=True)
    
    def split_df(self, df):
        """
            Creates two dataframes from a CSV file, one ready for clustering with additional columns removed and the other with all columns left intact.
        """

        df1 = df.copy()

        df1 = df1.dropna()
        df2_clear = df1.copy()
        df2_clear = df2_clear.drop(columns=["system:index", ".geo"])
        df2_clear = df2_clear.set_index("file_name")
        df1 = df1.set_index("file_name")
        df2_clear = df2_clear.apply(pd.to_numeric, errors='coerce')

        return df1, df2_clear
    
    def create_subregions():
        """
            Creates subregions based on the polygons in the DataFrame.
            This method should be implemented in subclasses.
        """
        raise NotImplementedError("Subclasses should implement this method.")
    
    def create_subregions(chunks, sift_percentage=0.5):
        """
            Shifts all polygons to 8 positions based on half the length in longitude and latitude, to create subregions - one for each direction from the center.

            Assumes aoi contains all similar polygons, and are similar to a square.

            Parameters:
                aoi (GeoDataFrame): The area of interest containing geometries.
                sift_percentage (float): Percentage of the length to shift the center point. Default is 0.5 (50%) for half the length.
        """

        if chunks.empty:
            return {}
        
        first_polygon = chunks.geometry.iloc[0]

        # take top two points of the polygon and get the length between them
        top_points = first_polygon.exterior.coords[:2]
        length_lon = abs(top_points[0][0] - top_points[1][0])

        # calculate the shift amount
        shift_amount = length_lon * sift_percentage

        shift_directions = {
            "left": (-shift_amount, 0),
            "right": (shift_amount, 0),
            "up": (0, shift_amount),
            "down": (0, -shift_amount),
            "top_left": (-shift_amount, shift_amount),
            "top_right": (shift_amount, shift_amount),
            "bottom_left": (-shift_amount, -shift_amount),
            "bottom_right": (shift_amount, -shift_amount),
        }

        subregions = []

        # Create subregions by shifting the geometries in all directions
        for _, (dx, dy) in shift_directions.items():
            gdf_shifted = chunks.copy()
            gdf_shifted["geometry"] = gdf_shifted["geometry"].translate(dx, dy)
            subregions.append(gdf_shifted)

        # Combine all into one GeoDataFrame
        subregions = pd.concat(subregions, ignore_index=True)
        return gpd.GeoDataFrame(subregions, crs=chunks.crs)
    
    def convert_df_to_geodf(df, geo_col='.geo', crs="EPSG:32638"):
        """
        Converts a DataFrame with a '.geo' column (GeoJSON strings) to a GeoDataFrame.
        """
        df = df.copy()

        if df.empty:
            return gpd.GeoDataFrame(columns=df.columns.tolist(), crs=crs)
        if df is type(gpd.DataFrame):
            return df 

        # Only parse if the entry is a string
        def safe_parse(x):
            if isinstance(x, str):
                try:
                    return shape(json.loads(x))
                except Exception as e:
                    print(f"[WARNING] Bad geometry skipped: {x[:30]}... ({e})")
            return None

        df['geometry'] = df[geo_col].apply(safe_parse)
        df = df[df['geometry'].notnull()]  # drop invalid rows

        return gpd.GeoDataFrame(df, geometry='geometry', crs=crs)

    def build_row_labels(self):
        """
        Builds a DataFrame with labelled polygons based on the points in the GeoDataFrame.

        Args:
            df (GeoDataFrame): The GeoDataFrame containing the points.
            labels (GeoDataFrame): The GeoDataFrame containing the polygons and their labels.

        Returns:
            pd.DataFrame: A DataFrame with the labels for each point.
        """


        # Prepare and Ensure in GeoDataFrame format
        df_gdf = self.data.copy()
        if df_gdf is type(pd.DataFrame):
            df_gdf = self.convert_df_to_geodf(df_gdf, geo_col='.geo', crs=f"EPSG:{self.UTM_ESPG}")
        labels_gdf = self.points.copy()

        # check for intersections and assign labels
        df_gdf['label'] = None
        for _, label_row in labels_gdf.iterrows():
            print(f"progress: {_}/{len(labels_gdf)} labels processed.")
            label = label_row['label']
            
            # Check each point until assigned or dropped
            for idx, point in df_gdf.iterrows():
                if label_row.geometry.intersects(point.geometry):
                    df_gdf.at[idx, 'class'] = label
                    break

        self.gdf_labels = df_gdf

        return df_gdf
        
    def fit(self, cluster_fn, index_column='file_name'):
        """
            Fits the clustering model to the data.
            This method should be implemented in subclasses.
        """

        subregions_data = []

        for _, subregion in self.subregions.iterrows():
            # clip the data to the subregion
            # based on .geo column drop all rows that do not intersect with the polygon

            df_clipped = self.clip_dataframe(subregion.geometry, self.data)
            df_clipped = df_clipped.dropna()
            df_clipped = df_clipped.drop(columns=["system:index", ".geo"]) # unessusary columns for clustering
            # df_clipped = df_clipped.set_index("file_name")
            df_clipped = df_clipped.apply(pd.to_numeric, errors='coerce')

            if df_clipped.empty:
                print("No data points found in this subregion.")
                continue

            subregions_data.append(df_clipped)

        matrix = pd.DataFrame()
        index_number = 0

        for subregion_df in subregions_data:
            print(f"Clustering {len(subregion_df)} points from subregion.")

            # Perform clustering
            labels, indecies = kmeans_clustering(subregion_df, k=10)

            col_name = f'cluster_label_{index_number}'

            # Temporary DataFrame
            temp_df = pd.DataFrame({col_name: labels, 'file_name': indecies})

            # Group in case of duplicates
            temp_grouped = temp_df.groupby('file_name')[col_name].first()

            # Convert to DataFrame for merging
            temp_grouped = temp_grouped.to_frame()

            print(f"temp columns: {temp_grouped.columns}")

            # Merge with main matrix
            matrix = matrix.join(temp_grouped, how='outer') if not matrix.empty else temp_grouped

            index_number += 1
            print(len(matrix.columns), "columns in the matrix after clustering.")
            print(len(matrix), "rows in the matrix after clustering.")

        matrix = matrix.join(self.data[[".geo"]], how='outer')
        self.df_sparse = matrix

        return matrix
        

In [ ]:
from collections import Counter


def convert_to_map(df_sparse, df_data, gdf_labelled):
    """
    Converts the sparse DataFrame of multiple cluster label columns to a single-column label prediction
    based on majority voting from known labels.

    Args:
        df_sparse (pd.DataFrame): DataFrame with 'cluster_label_*' columns and index as file_name.
        df_data (pd.DataFrame): Original data containing '.geo' and any other metadata.
        gdf_labelled (GeoDataFrame): GeoDataFrame with true labels and index as file_name.

    Returns:
        pd.DataFrame: DataFrame indexed by file_name, with predicted label (or None if unassignable).
    """

    # Ensure labels column exists
    if 'class' not in gdf_labelled.columns:
        raise ValueError("gdf_labelled must contain a 'class' column with ground truth labels.")

    predictions = pd.Series(index=df_sparse.index, dtype=object)

    for col in df_sparse.columns:
        if not col.startswith("cluster_label_"):
            continue

        # Get cluster IDs and their associated true labels
        cluster_ids = df_sparse[col]
        known_labels = gdf_labelled['class']

        # Build mapping: cluster_id -> list of known labels
        cluster_to_labels = {}
        for file_name, cluster_id in cluster_ids.items():
            if file_name in known_labels and pd.notna(known_labels[file_name]):
                cluster_to_labels.setdefault(cluster_id, []).append(known_labels[file_name])

        # Compute majority label for each cluster
        cluster_to_majority = {}
        for cluster_id, labels in cluster_to_labels.items():
            if not labels:
                cluster_to_majority[cluster_id] = None
            else:
                label_counts = Counter(labels)
                most_common = label_counts.most_common()
                top_label = most_common[0][0] if len(most_common) == 1 or most_common[0][1] != most_common[1][1] \
                    else np.random.choice([l for l, c in most_common if c == most_common[0][1]])
                cluster_to_majority[cluster_id] = top_label

        # Assign predicted label per row
        for idx in df_sparse.index:
            cluster_id = df_sparse.at[idx, col]
            label = cluster_to_majority.get(cluster_id, None)
            if pd.isna(predictions.at[idx]) and label is not None:
                predictions.at[idx] = label

    # Create output DataFrame
    result_df = pd.DataFrame({'predicted_label': predictions})

    # Join metadata like `.geo` if needed
    if '.geo' in df_data.columns:
        result_df = result_df.join(df_data['.geo'])

    return result_df

In [225]:
c = cluster(chunks, df1, points=labels)
matrix = c.fit(kmeans_clustering)
c.build_row_labels()

Clustering 61842 points from subregion.
temp columns: Index(['cluster_label_0'], dtype='object')
1 columns in the matrix after clustering.
61842 rows in the matrix after clustering.
Clustering 62078 points from subregion.
temp columns: Index(['cluster_label_1'], dtype='object')
2 columns in the matrix after clustering.
123920 rows in the matrix after clustering.
Clustering 61916 points from subregion.
temp columns: Index(['cluster_label_2'], dtype='object')
3 columns in the matrix after clustering.
185836 rows in the matrix after clustering.
Clustering 62085 points from subregion.
temp columns: Index(['cluster_label_3'], dtype='object')
4 columns in the matrix after clustering.
247921 rows in the matrix after clustering.
Clustering 59799 points from subregion.
temp columns: Index(['cluster_label_4'], dtype='object')
5 columns in the matrix after clustering.
307720 rows in the matrix after clustering.
Clustering 61479 points from subregion.
temp columns: Index(['cluster_label_5'], dtype

,system:index,0_B1,0_B11,0_B12,0_B2,0_B3,0_B4,0_B5,0_B6,0_B7,...,9_B5,9_B6,9_B7,9_B8,9_B8A,9_B9,.geo,geometry,label,class
file_name,,,,,,,,,,,,,,,,,,,,,
tile_43997486_34112017_43998137_34112113,1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_...,769.925866,3322.198887,2644.624125,1128.620894,1753.336923,2010.568300,2634.353258,3302.266379,3455.818704,...,3648.235249,3713.545965,3826.787522,3871.171013,3920.213786,3837.507039,"{""type"":""Polygon"",""coordinates"":[[[43.99813739...","POLYGON ((43.99813739573809 34.11211410383254,...",None,NaN
tile_44104382_34112843_44105034_34112938,1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_...,298.404905,2157.473569,1582.494005,528.488556,831.356676,934.839237,1352.791826,1881.520708,2069.254223,...,1236.062398,2438.234332,2861.378801,3031.939292,3114.423978,2432.149682,"{""type"":""Polygon"",""coordinates"":[[[44.10438435...","POLYGON ((44.10438435337108 34.11293465494744,...",None,NaN
tile_43998136_34112022_43999221_34112122,1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_...,528.454970,2495.875662,1713.649631,640.474810,1152.119767,1171.487171,1786.087047,3059.746338,3359.432014,...,2445.911989,2515.677113,2636.289053,2605.584472,2756.973423,3091.114321,"{""type"":""Polygon"",""coordinates"":[[[43.99813739...","POLYGON ((43.99813739573809 34.11211410383254,...",None,NaN
tile_43999220_34112031_44000305_34112130,1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_...,335.551290,2071.815893,1344.805779,429.478019,894.811249,770.613622,1463.251290,2839.394324,3180.367699,...,2060.790668,2105.297921,2202.409155,2225.273773,2299.559797,2382.651482,"{""type"":""Polygon"",""coordinates"":[[[43.99922097...",POLYGON ((43.99922097074953 34.112123037328075...,None,NaN
tile_44000304_34112040_44001389_34112139,1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_...,285.086948,1878.116894,1177.896530,406.417493,840.937113,721.051941,1302.629802,2778.296159,3145.126910,...,2059.086564,2108.126792,2208.764339,2278.053815,2330.043238,2451.489202,"{""type"":""Polygon"",""coordinates"":[[[44.00030452...","POLYGON ((44.00030452142709 34.11213195754417,...",None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
tile_43910796_34410765_43911896_34411676,1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_...,668.574031,3216.564180,2568.033046,1062.455223,1676.717285,1928.949963,2485.217906,3227.529617,3400.955893,...,3257.947338,3345.804999,3482.857085,3577.533450,3578.926340,3614.595852,"{""type"":""Polygon"",""coordinates"":[[[43.91080588...","POLYGON ((43.910805889628 34.41076691363211, 4...",None,NaN
tile_43910784_34411666_43911884_34412578,1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_...,598.349524,2971.679810,2268.055660,864.023121,1434.253042,1580.016358,2230.677262,3098.499345,3286.429584,...,2963.722553,3035.604837,3159.457128,3250.330399,3288.109756,3387.261634,"{""type"":""Polygon"",""coordinates"":[[[43.91079693...","POLYGON ((43.91079693382084 34.41166767820974,...",None,NaN
tile_43910773_34412568_43911872_34413479,1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_...,323.005126,2383.341164,1565.403920,463.461825,965.500234,888.312974,1656.546377,3127.376917,3492.560795,...,2529.101203,2555.877340,2659.343720,2718.644720,2766.098965,2802.713800,"{""type"":""Polygon"",""coordinates"":[[[43.91078360...","POLYGON ((43.9107836093242 34.41256834144177, ...",None,NaN


In [227]:
# get predictions
predictions = convert_to_map(c.df_sparse, c.data, c.gdf_labels)

# Save predictions to a CSV file
output_path = "predictions.csv"
predictions.to_csv(output_path, index=True)

In [232]:
predictions.head()

,predicted_label,.geo,numeric_label
file_name,,,
tile_43637469_34558614_43638574_34559528,NaN,"{""type"":""Polygon"",""coordinates"":[[[43.63748527...",0
tile_43637484_34557712_43638589_34558626,NaN,"{""type"":""Polygon"",""coordinates"":[[[43.63749867...",0
tile_43637499_34556811_43638603_34557725,NaN,"{""type"":""Polygon"",""coordinates"":[[[43.63751207...",0
tile_43637514_34555909_43638618_34556823,NaN,"{""type"":""Polygon"",""coordinates"":[[[43.63752986...",0
tile_43637528_34555008_43638633_34555922,Urban,"{""type"":""Polygon"",""coordinates"":[[[43.63754326...",3


In [231]:
# mapping dict for labels to numbers
mapping = {
    None: 0,
    np.NaN: 0,
    "NaN": 0,
    "None": 0,
    "Water": 1,
    "Agricultural": 2,
    "Urban": 3
}

# add numeric labels to predictions
predictions['numeric_label'] = predictions['predicted_label'].map(mapping)

# Save to CSV
output_path = "predictions_with_numeric_labels.csv"
predictions.to_csv(output_path, index=True)

In [ ]:
# generate tif 
def os_path_chcker(output_dir, postfix=".tif", NAME_CODE_LIM=8, FLAG_APPEND_POSTFIX=True):
    """
        Ensures that the output directory is valid, exists and can be written to.

        This is a simple function that does three things:
            1. Checks if the directory exists, if not creates it.
            2. Checks if the file name is valid (and not empty, else creates a unique filename).
            3. (Optional) Appends a postfix to the file name if it does not already end with it.

    Args:
        output_dir (str): The directory path to check.
        postfix (str, optional): The postfix to append to the file name if it does not already end with it. Default is ".tif".
        NAME_CODE_LIM (int, optional): The length of the hex code to generate for filename. Default is 8.
        FLAG_APPEND_POSTFIX (bool, optional): Whether to append the postfix if the file name does not end with it. Default is True.
    """


    # path existance check
    if not os.path.exists(os.path.dirname(output_dir)):
        os.makedirs(os.path.dirname(output_dir))
        print(f"Created directory: {os.path.dirname(output_dir)}")


    # file name existance check
    flag_filename = not os.path.basename(output_dir)
    while flag_filename:
        print("Generating new file name...")
        # assign unique name to the tif file
        new_file_name = f"{os.urandom(NAME_CODE_LIM).hex()}{postfix}"
        if not os.path.exists(output_dir + new_file_name):
            output_dir += new_file_name
            flag_filename = False
            print(f"New File Name: {output_dir}")


    # check labelled correctly 
    if FLAG_APPEND_POSTFIX and not output_dir.endswith(postfix):
        print(f"Output directory {output_dir} does not end with .tif, appending .tif")
        output_dir += {postfix}
        
    return output_dir

def export_to_tif(df, bands, output_dir, res=50, UTM_ESPG=32638, EPSG=4326):
    """
        Exports a DataFrame generated by PlotToSat to a GeoTIFF file. 

        Note: df files MUST CONTAIN:
        - `.geo` column with GeoJSON geometries.
        - `file_name` column with unique identifiers for each row.
        - 1 Unique band labelled in `bands` list - this CAN BE LABELLED CLUSTERS. 

        Expected formats:
            Time-Series data should be in a DataFrame with 'month_band' columns (e.g., '0_B1', '1_B2', etc.).
            if a band is provideda e.g.m B1 but there are no pre-fix values this data will be assumed to be a single band AS-IS, and will be included in the output just as that band alone. 

        Args:
            df (pd.DataFrame): PlotToSat Style pandas DataFrame containing time-series data or single band data. Must have file_name and `.geo` geometry columns.
            bands (list): List of band names to include in the output. e.g., ['B1', 'B2', 'B3'] or ['SingleBand', etc.] can be mixed with single band data.
            output_dir (str): Output file directory for the GeoTIFF.
            res (int, optional): Resolution of the output raster in meters. Default is 50m.
            UTM_ESPG (int, optional): EPSG code for the UTM coordinate reference system. Default is 32638.
            


            # EPSG (int, optional): EPSG code for the coordinate reference system.
    """

    # Ensure output directory is valid, exists and can be written to.
    file = os_path_chcker(output_dir, postfix=".tif", NAME_CODE_LIM=8, FLAG_APPEND_POSTFIX=True)


    # Sort the Bands into single and time-series band data. 
    column_heads = df.columns.tolist()

    
    # all available bands in the data
    band_columns = [col for col in column_heads if any(col.endswith(band) for band in bands)]


    # Acceptable bands to process 
    acceptable_lst = []
    for col in bands:
        if any(col.startswith(f"{i}_") for i in range(12)):
            # must be non-prefix and only one band of that ts 
            if col not in acceptable_lst and any(coli for coli in band_columns if coli == col):
                acceptable_lst.append(col)
            # acceptable_lst.append(col)
        else:
            # must exist in band_columns and take all 
            appended = False
            for i in range(12):
                if any(coli for coli in band_columns if coli == f"{i}_{col}"):
                    if f"{i}_{col}" not in acceptable_lst:
                        # add only if not already in list     
                        acceptable_lst.append(f"{i}_{col}")
                        appended = True

            if appended == False:
                # if not appended must be a unique column 
                acceptable_lst.append(col)
    
    band_columns = acceptable_lst

 
    geometry = df[".geo"].apply(lambda x: shape(json.loads(x)))
    gdf = gpd.GeoDataFrame(df[band_columns].copy(), geometry=geometry, crs=f"EPSG:{EPSG}") # EPSG not UTM_ESPG else will raise an error

    gdf_utm = gdf.to_crs(epsg=UTM_ESPG)

    minx, miny, maxx, maxy = gdf_utm.total_bounds
    width = int((maxx - minx) / res)
    height = int((maxy - miny) / res)

    transform = from_origin(minx, maxy, res, res)

    if width <= 0 or height <= 0:
        raise ValueError(f"Invalid raster dimensions (width={width}, height={height}). Check CRS and resolution.")

    # Build Raster Stack
    rasters = []
    for band in band_columns:
        values = gdf_utm[band]
        shapes = ((geom, val) for geom, val in zip(gdf_utm.geometry, values))
        raster = rasterize(
            shapes=shapes,
            out_shape=(height, width),
            transform=transform,
            dtype="float32",
            fill=np.NaN,  # Fill with NaN values for NO FUCKING DATA
        )

        rasters.append(raster)

    raster_stack = raster_stack = np.stack(rasters, axis=0) if len(rasters) >= 2 else rasters

    # Save rasters to GeoTIFF 
    with rasterio.open(
        file,
        "w",
        driver="GTiff",
        height=height,
        width=width,
        count=len(band_columns),
        dtype="float32",
        crs=f"EPSG:{UTM_ESPG}",
        transform=transform,
        nodata=np.NaN,  # Set NoData value to NaN
    ) as dst:
        for i, band in enumerate(band_columns):
            dst.write(raster_stack[i], i + 1)
            dst.set_band_description(i + 1, band) # Keep band discription!!!



# predictions to tif
output_dir = "/predictions.tif"
export_to_tif(predictions, bands=["numeric_label"], output_dir=output_dir, res=50, UTM_ESPG=32638, EPSG=4326)

In [ ]:
def build_map(self):
    """
        Builds a map with the clustered points and labels.

        Args:
            voting_style_chunk (str): The voting style for clustering. Default is "majority".

        Returns:
            pandas.DateFrame: A DataFrame with each tile with relevent labels. 
                    -1 if no label is assigned ever.
                    else 'label' if a label is assigned.
    """

    if self.df_sparse.empty:
        raise ValueError("DataFrame is empty. Please run fit() method first.")
    
    if self.gdf_labels is None:
        print("Building row labels... this may take a while.")
        self.build_row_labels()

    labels_gdf = self.gdf_labels.copy()
    sparse_df = self.df_sparse.copy()

    # for each column create seperate dataframe with label column, and cluster label column
    for col in sparse_df.columns:
        if col.startswith("cluster_label_"):
            # create new dataframe with only this column + class from labels_gdf (make standard pandas DataFrame)
            temp_df = pd.DataFrame({
                'file_name': sparse_df.index,
                'cluster_label': sparse_df[col]
                'class': labels_gdf['class']
            })

            # for each cluster count use of each label and assign the most common label to the cluster via majority vote

            # for each cluster count use of each label and assign the most common label to the cluster
            temp_df = temp_df.groupby('cluster_label')['class'].agg(lambda x: x.mode()[0] if not x.mode().empty else -1).reset_index()

            # vote style should be columns cluster_{label}, ... with a 1 or 0 for each row 
            for label in labels_gdf['class'].unique():
                temp_df[f'cluster_{label}'] = (temp_df['class'] == label)

In [219]:
c.df_sparse.head()

,cluster_label_0,cluster_label_1,cluster_label_2,cluster_label_3,cluster_label_4,cluster_label_5,.geo
file_name,,,,,,,
tile_43637469_34558614_43638574_34559528,NaN,NaN,NaN,NaN,NaN,NaN,"{""type"":""Polygon"",""coordinates"":[[[43.63748527..."
tile_43637484_34557712_43638589_34558626,NaN,NaN,NaN,NaN,NaN,NaN,"{""type"":""Polygon"",""coordinates"":[[[43.63749867..."
tile_43637499_34556811_43638603_34557725,NaN,NaN,NaN,NaN,NaN,NaN,"{""type"":""Polygon"",""coordinates"":[[[43.63751207..."
tile_43637514_34555909_43638618_34556823,NaN,NaN,NaN,NaN,NaN,NaN,"{""type"":""Polygon"",""coordinates"":[[[43.63752986..."
tile_43637528_34555008_43638633_34555922,NaN,NaN,NaN,NaN,5.0,NaN,"{""type"":""Polygon"",""coordinates"":[[[43.63754326..."


In [155]:
len(df1)

372313

In [ ]:
# c = cluster(chunks, df1)
# matrix = c.fit(kmeans_clustering)

Clustering 61842 points from subregion.
temp columns: Index(['cluster_label_0'], dtype='object')
1 columns in the matrix after clustering.
61842 rows in the matrix after clustering.
Clustering 62078 points from subregion.
temp columns: Index(['cluster_label_1'], dtype='object')
2 columns in the matrix after clustering.
123920 rows in the matrix after clustering.
Clustering 61916 points from subregion.
temp columns: Index(['cluster_label_2'], dtype='object')
3 columns in the matrix after clustering.
185836 rows in the matrix after clustering.
Clustering 62085 points from subregion.
temp columns: Index(['cluster_label_3'], dtype='object')
4 columns in the matrix after clustering.
247921 rows in the matrix after clustering.
Clustering 59799 points from subregion.
temp columns: Index(['cluster_label_4'], dtype='object')
5 columns in the matrix after clustering.
307720 rows in the matrix after clustering.
Clustering 61479 points from subregion.
temp columns: Index(['cluster_label_5'], dtype

In [ ]:
def convert_df_to_geodf(df, geo_col='.geo', crs="EPSG:32638"):
    """
    Converts a DataFrame with a '.geo' column (GeoJSON strings) to a GeoDataFrame.
    """
    df = df.copy()

    # Only parse if the entry is a string
    def safe_parse(x):
        if isinstance(x, str):
            try:
                return shape(json.loads(x))
            except Exception as e:
                print(f"[WARNING] Bad geometry skipped: {x[:30]}... ({e})")
        return None

    df['geometry'] = df[geo_col].apply(safe_parse)
    df = df[df['geometry'].notnull()]  # drop invalid rows

    return gpd.GeoDataFrame(df, geometry='geometry', crs=crs)

def check_intersection(poly, point):
    """
    Checks if a point intersects with a polygon.

    Args:
        poly (shapely.geometry.Polygon): The polygon to check against.
        point (shapely.geometry.Point): The point to check.

    Returns:
        bool: True if the point intersects with the polygon, False otherwise.
    """
    return poly.intersects(point)

def build_row_labels(df, labels, utm_epsg=32638):
    """
    Builds a DataFrame with labelled polygons based on the points in the GeoDataFrame.

    Args:
        df (GeoDataFrame): The GeoDataFrame containing the points.
        labels (GeoDataFrame): The GeoDataFrame containing the polygons and their labels.

    Returns:
        pd.DataFrame: A DataFrame with the labels for each point.
    """


    df_gdf = convert_df_to_geodf(df, geo_col='.geo', crs=f"EPSG:{utm_epsg}")    
    labels_gdf = labels.copy()

    # check for intersections and assign labels
    df_gdf['label'] = None
    for _, label_row in labels_gdf.iterrows():
        print(f"progress: {_}/{len(labels_gdf)} labels processed.")
        poly = label_row.geometry
        label = label_row['label']
        
        # Check each point in the DataFrame
        for idx, point in df_gdf.iterrows():
            if check_intersection(poly, point.geometry):
                df_gdf.at[idx, 'class'] = label
                # next as cannot intersect multiple polygons
                break

    return df_gdf

build_row_labels(matrix, labels).head()


progress: 0/141 labels processed.
progress: 1/141 labels processed.
progress: 2/141 labels processed.
progress: 3/141 labels processed.
progress: 4/141 labels processed.
progress: 5/141 labels processed.
progress: 6/141 labels processed.
progress: 7/141 labels processed.
progress: 8/141 labels processed.
progress: 9/141 labels processed.
progress: 10/141 labels processed.
progress: 11/141 labels processed.
progress: 12/141 labels processed.
progress: 13/141 labels processed.
progress: 14/141 labels processed.
progress: 15/141 labels processed.
progress: 16/141 labels processed.
progress: 17/141 labels processed.
progress: 18/141 labels processed.
progress: 19/141 labels processed.
progress: 20/141 labels processed.
progress: 21/141 labels processed.
progress: 22/141 labels processed.
progress: 23/141 labels processed.
progress: 24/141 labels processed.
progress: 25/141 labels processed.
progress: 26/141 labels processed.
progress: 27/141 labels processed.
progress: 28/141 labels proces

,cluster_label_0,cluster_label_1,cluster_label_2,cluster_label_3,cluster_label_4,cluster_label_5,.geo,class,geometry,label
file_name,,,,,,,,,,
tile_43637469_34558614_43638574_34559528,NaN,NaN,NaN,NaN,NaN,NaN,"{""type"":""Polygon"",""coordinates"":[[[43.63748527...",NaN,"POLYGON ((43.637 34.559, 43.639 34.559, 43.639...",None
tile_43637484_34557712_43638589_34558626,NaN,NaN,NaN,NaN,NaN,NaN,"{""type"":""Polygon"",""coordinates"":[[[43.63749867...",NaN,"POLYGON ((43.637 34.558, 43.639 34.558, 43.639...",None
tile_43637499_34556811_43638603_34557725,NaN,NaN,NaN,NaN,NaN,NaN,"{""type"":""Polygon"",""coordinates"":[[[43.63751207...",NaN,"POLYGON ((43.638 34.557, 43.639 34.557, 43.639...",None
tile_43637514_34555909_43638618_34556823,NaN,NaN,NaN,NaN,NaN,NaN,"{""type"":""Polygon"",""coordinates"":[[[43.63752986...",NaN,"POLYGON ((43.638 34.556, 43.639 34.556, 43.639...",None
tile_43637528_34555008_43638633_34555922,NaN,NaN,NaN,NaN,5.0,NaN,"{""type"":""Polygon"",""coordinates"":[[[43.63754326...",NaN,"POLYGON ((43.638 34.555, 43.639 34.555, 43.639...",None


In [184]:
def build_votes(matrix, labels):
    """
        Builds a voting system based on the clustered matrix.
        This method should be implemented in subclasses.
    """

    matrix_2 = matrix.copy()

    # remove .geo columns that are empty
    matrix_2 = matrix_2[matrix_2['.geo'].notnull()]
    matrix_2 = matrix_2[matrix_2['.geo'] != '']

    matrix_2 = matrix_2['.geo'].apply(lambda x: shape(json.loads(x)))
    
    return matrix_2



    classes = labels['label'].unique()

    # append new column and fill with NaNs
    # ".class" can be either "Water", "Agricultural", "Urban" or "None"
    matrix_2["class"] = np.nan

    # check for geometry intersection with the point and polygon of each point in labels with the .geo column in the matrix
    for index, row in labels.iterrows():
        point_geom = shape(json.loads(row['.geo']))
        point_class = row['label']

        # find the closest point in the matrix
        closest_index = matrix_2.geometry.apply(lambda x: x.intersects(point_geom)).idxmax()

        if closest_index is not None:
            # assign the class to the closest point
            matrix_2.at[closest_index, "class"] = point_class

    return matrix_2

    # print(f"Classes found: {classes}")

            # for each point 


build_row_labels(matrix, labels).head()

# # show full matrix scrollable 
# matrix

AttributeError: 'Series' object has no attribute 'geometry'

In [95]:
def clip_dataframe(polygon, df):
    """
        Clips the DataFrame based on the .geo column and a given polygon.   
    
    Args:
        polygon (shapely.geometry.Polygon): The polygon to clip the DataFrame to.
        df (pd.DataFrame): The DataFrame containing the geometries to be clipped.
        
    Returns:
        pd.DataFrame: The clipped DataFrame.
    """

    # based on .geo column drop all rows that do not intersect with the polygon
    df['geometry'] = df['.geo'].apply(lambda x: shape(json.loads(x)))
    gdf = gpd.GeoDataFrame(df, geometry='geometry', crs=32638)
    gdf = gdf[gdf.geometry.within(polygon)]
    gdf = gdf.drop(columns=['geometry'])

    # convert back to DataFrame
    df_clipped = pd.DataFrame(gdf)

    return df_clipped

In [ ]:
df1.head()

,system:index,0_B1,0_B11,0_B12,0_B2,0_B3,0_B4,0_B5,0_B6,0_B7,...,9_B2,9_B3,9_B4,9_B5,9_B6,9_B7,9_B8,9_B8A,9_B9,.geo
file_name,,,,,,,,,,,,,,,,,,,,,
tile_43997486_34112017_43998137_34112113,1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_...,769.925866,3322.198887,2644.624125,1128.620894,1753.336923,2010.568300,2634.353258,3302.266379,3455.818704,...,1855.620740,2541.523502,3334.350796,3648.235249,3713.545965,3826.787522,3871.171013,3920.213786,3837.507039,"{""type"":""Polygon"",""coordinates"":[[[43.99813739..."
tile_44104382_34112843_44105034_34112938,1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_...,298.404905,2157.473569,1582.494005,528.488556,831.356676,934.839237,1352.791826,1881.520708,2069.254223,...,547.670409,801.668174,662.603379,1236.062398,2438.234332,2861.378801,3031.939292,3114.423978,2432.149682,"{""type"":""Polygon"",""coordinates"":[[[44.10438435..."
tile_43998136_34112022_43999221_34112122,1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_...,528.454970,2495.875662,1713.649631,640.474810,1152.119767,1171.487171,1786.087047,3059.746338,3359.432014,...,1154.201264,1593.018742,2124.370565,2445.911989,2515.677113,2636.289053,2605.584472,2756.973423,3091.114321,"{""type"":""Polygon"",""coordinates"":[[[43.99813739..."
tile_43999220_34112031_44000305_34112130,1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_...,335.551290,2071.815893,1344.805779,429.478019,894.811249,770.613622,1463.251290,2839.394324,3180.367699,...,1018.777621,1394.102079,1854.067183,2060.790668,2105.297921,2202.409155,2225.273773,2299.559797,2382.651482,"{""type"":""Polygon"",""coordinates"":[[[43.99922097..."
tile_44000304_34112040_44001389_34112139,1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_...,285.086948,1878.116894,1177.896530,406.417493,840.937113,721.051941,1302.629802,2778.296159,3145.126910,...,1015.020712,1386.387060,1850.829881,2059.086564,2108.126792,2208.764339,2278.053815,2330.043238,2451.489202,"{""type"":""Polygon"",""coordinates"":[[[44.00030452..."


In [103]:
subregions = chunks

subregions_data = []
for _, subregion in subregions.iterrows():
    # clip the data to the subregion
    # based on .geo column drop all rows that do not intersect with the polygon

    df_clipped = clip_dataframe(subregion.geometry, df1)
    df_clipped = df_clipped.dropna()
    df_clipped = df_clipped.drop(columns=["system:index", ".geo"])
    # df_clipped = df_clipped.set_index("file_name")
    df_clipped = df_clipped.apply(pd.to_numeric, errors='coerce')

    if df_clipped.empty:
        print("No data points found in this subregion.")
        continue  

    print(f"Clipping {len(df_clipped)} points from subregion.")

    subregions_data.append(df_clipped)

# # clean 
#         df1 = df1.dropna()
#         df2_clear = df1.copy()
#         df2_clear = df2_clear.drop(columns=["system:index", ".geo"])
#         df2_clear = df2_clear.set_index("file_name")
#         df1 = df1.set_index("file_name")
#         df2_clear = df2_clear.apply(pd.to_numeric, errors='coerce')

Clipping 62085 points from subregion.
Clipping 62078 points from subregion.
Clipping 61479 points from subregion.
Clipping 61916 points from subregion.
Clipping 61842 points from subregion.
Clipping 59799 points from subregion.


In [113]:
def kmeans_clustering(df, k=10):
    """
        Applies KMeans clustering to the DataFrame and returns the cluster labels.
        
        Args:
            df (pd.DataFrame): The DataFrame containing the data to be clustered.
            n_clusters (int): The number of clusters to form. Default is 10.
        
        Returns:
            np.ndarray: An array of cluster labels for each point in the DataFrame.
    """
    kmeans = KMeans(n_clusters=k, random_state=42) # TODO: make random_state random
    kmeans.fit(df)
    
    return kmeans.labels_, df.index.tolist()

matrix = pd.DataFrame()
index_number = 0

for subregion_df in subregions_data:
    print(f"Clustering {len(subregion_df)} points from subregion.")

    # Perform clustering
    labels, indecies = kmeans_clustering(subregion_df, k=10)

    col_name = f'cluster_label_{index_number}'

    # Temporary DataFrame
    temp_df = pd.DataFrame({col_name: labels, 'file_name': indecies})

    # Group in case of duplicates
    temp_grouped = temp_df.groupby('file_name')[col_name].first()

    # Convert to DataFrame for merging
    temp_grouped = temp_grouped.to_frame()

    # Merge with main matrix
    matrix = matrix.join(temp_grouped, how='outer') if not matrix.empty else temp_grouped

    index_number += 1
#     # sklearn kmeans clustering
#     for k in [10, 15, 20, 25, 30, 40, 50]:
#         kmeans = KMeans(n_clusters=k, random_state=random_state)
#         kmeans.fit(df_raw)

#         # disp_labels(df, kmeans.labels_, output_img=f"{output_img}_k{k}.png", output_tif=f"{output_tif}_k{k}.tif")
#         disp_labels(df, kmeans.labels_, output_img=f"{output_img}_k{k}.png", output_tif=f"{output_tif}_k{k}.tif")

Clustering 62085 points from subregion.
Clustering 62078 points from subregion.
Clustering 61479 points from subregion.
Clustering 61916 points from subregion.
Clustering 61842 points from subregion.
Clustering 59799 points from subregion.


In [112]:
matrix

,cluster_label_0,cluster_label_1,cluster_label_2,cluster_label_3,cluster_label_4,cluster_label_5
file_name,,,,,,
tile_43637528_34555008_43638633_34555922,NaN,NaN,NaN,NaN,NaN,5.0
tile_43637572_34552303_43638677_34553217,NaN,NaN,NaN,NaN,NaN,9.0
tile_43637587_34551402_43638691_34552315,NaN,NaN,NaN,NaN,NaN,9.0
tile_43637631_34548697_43638735_34549611,NaN,NaN,NaN,NaN,NaN,9.0
tile_43637675_34545992_43638780_34546906,NaN,NaN,NaN,NaN,NaN,9.0
...,...,...,...,...,...,...
tile_44187733_33898871_44188823_33899780,NaN,1.0,NaN,NaN,NaN,NaN
tile_44187742_33897969_44188832_33898878,NaN,1.0,NaN,NaN,NaN,NaN
tile_44187751_33897067_44188841_33897976,NaN,4.0,NaN,NaN,NaN,NaN
